In [1]:
pwd

'C:\\Users\\a.alatriste.trujillo\\dl_udacity'

# Deep Learning
## Assignment 4

Previously in 2_fullyconnected.ipynb and 3_regularization.ipynb, we trained fully connected networks to classify notMNIST characters.
The goal of this assignment is make the neural network convolutional.

In [2]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle
from six.moves import range

#### load pickle file

In [3]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


#### Reformat into a TensorFlow-friendly shape:
convolutions need the image data formatted as a cube (width by height by #channels)
labels as float 1-hot encodings.

In [4]:
image_size = 28
num_labels = 10
num_channels = 1 # grayscale

import numpy as np

def reformat(dataset, labels):
  dataset = dataset.reshape(
    (-1, image_size, image_size, num_channels)).astype(np.float32)
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels

train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset,  test_labels  = reformat(test_dataset, test_labels)

print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28, 1) (200000, 10)
Validation set (10000, 28, 28, 1) (10000, 10)
Test set (10000, 28, 28, 1) (10000, 10)


In [5]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

#### create convolutional network

Let's build a small network with two convolutional layers, followed by one fully connected layer. Convolutional networks are more expensive computationally, so we'll limit its depth and number of fully connected nodes.

In [6]:
batch_size = 16
patch_size =  5
depth      = 16
num_hidden = 64

graph = tf.Graph()

with graph.as_default():

  # Input data.
  tf_train_dataset = tf.placeholder( tf.float32, shape=(batch_size, image_size, image_size, num_channels))
     
    
  tf_train_labels  = tf.placeholder( tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset  = tf.constant(test_dataset)
  
  # Variables.
  layer1_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, num_channels, depth], stddev=0.1))
  layer1_biases  = tf.Variable(tf.zeros([depth]))
  layer2_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, depth, depth], stddev=0.1))
  layer2_biases  = tf.Variable(tf.constant(1.0, shape=[depth]))
  layer3_weights = tf.Variable(tf.truncated_normal(
      [image_size // 4 * image_size // 4 * depth, num_hidden], stddev=0.1))
  layer3_biases  = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
  layer4_weights = tf.Variable(tf.truncated_normal(
      [num_hidden, num_labels], stddev=0.1))
  layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
  
  # Model.
  def model(data):
    conv = tf.nn.conv2d(data, layer1_weights, [1, 2, 2, 1], padding='SAME')
    hidden = tf.nn.relu(conv + layer1_biases)
    conv = tf.nn.conv2d(hidden, layer2_weights, [1, 2, 2, 1], padding='SAME')
    hidden = tf.nn.relu(conv + layer2_biases)
    shape = hidden.get_shape().as_list()
    reshape = tf.reshape(hidden, [shape[0], shape[1] * shape[2] * shape[3]])
    hidden = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
    return tf.matmul(hidden, layer4_weights) + layer4_biases
  
  # Training computation.
  logits = model(tf_train_dataset)
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
    
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
  test_prediction = tf.nn.softmax(model(tf_test_dataset))

#### run Conv net

In [7]:
num_steps = 1001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print('Initialized')
  for step in range(num_steps):
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run( [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 50 == 0):
      print('Minibatch loss at step %d: %f' % (step, l))
      print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 3.680835
Minibatch accuracy: 6.2%
Validation accuracy: 10.2%
Minibatch loss at step 50: 1.821053
Minibatch accuracy: 25.0%
Validation accuracy: 46.8%
Minibatch loss at step 100: 1.203637
Minibatch accuracy: 68.8%
Validation accuracy: 67.4%
Minibatch loss at step 150: 0.515840
Minibatch accuracy: 81.2%
Validation accuracy: 74.0%
Minibatch loss at step 200: 0.968173
Minibatch accuracy: 68.8%
Validation accuracy: 76.0%
Minibatch loss at step 250: 1.288327
Minibatch accuracy: 68.8%
Validation accuracy: 77.9%
Minibatch loss at step 300: 0.378076
Minibatch accuracy: 87.5%
Validation accuracy: 78.2%
Minibatch loss at step 350: 0.625042
Minibatch accuracy: 93.8%
Validation accuracy: 78.4%
Minibatch loss at step 400: 0.228576
Minibatch accuracy: 100.0%
Validation accuracy: 80.3%
Minibatch loss at step 450: 1.068870
Minibatch accuracy: 87.5%
Validation accuracy: 79.9%
Minibatch loss at step 500: 0.563505
Minibatch accuracy: 87.5%
Validation accuracy: 80.2%
M

___

#### Problem 1
The convolutional model above uses convolutions with stride 2 to reduce the dimensionality. Replace the strides by a max pooling operation (nn.max_pool()) of stride 2 and kernel size 2.


In [9]:
# max pooling layer:
pool_size = 2

In [10]:
# dense layer with ReLU activation:
def dense(x, W, b):
    z = tf.add(tf.matmul(x, W), b)
    a = tf.nn.relu(z)
    return a

# convolutional layer with ReLU activation:
def conv2d(x, W, b, stride_length=1):
    xW = tf.nn.conv2d(x, W, strides=[1, stride_length, stride_length, 1], padding='SAME')
    z = tf.nn.bias_add(xW, b)
    a = tf.nn.relu(z)
    return a

# max-pooling layer: 
def maxpooling2d(x, pool_size):
    return tf.nn.max_pool(x, 
                          ksize   = [1, pool_size, pool_size, 1], 
                          strides = [1, pool_size, pool_size, 1], 
                          padding = 'SAME')
# avg pooling layer
def avgpooling2d(x, pool_size):
    return tf.nn.avg_pool(x, 
                          ksize   = [1, pool_size, pool_size, 1], 
                          strides = [1, pool_size, pool_size, 1], 
                          padding = 'SAME' )

In [11]:
batch_size = 16
patch_size =  5
depth      = 16
num_hidden = 64

graph = tf.Graph()

with graph.as_default():

  # Input data.
  tf_train_dataset = tf.placeholder( tf.float32, shape=(batch_size, image_size, image_size, num_channels))
     
    
  tf_train_labels  = tf.placeholder( tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset  = tf.constant(test_dataset)
  
  # Variables.
  layer1_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, num_channels, depth], stddev=0.1))
  layer1_biases  = tf.Variable(tf.zeros([depth]))

  layer2_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, depth, depth], stddev=0.1))
  layer2_biases  = tf.Variable(tf.constant(1.0, shape=[depth]))

  layer3_weights = tf.Variable(tf.truncated_normal(
      [image_size // 4 * image_size // 4 * depth, num_hidden], stddev=0.1))
  layer3_biases  = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
  layer4_weights = tf.Variable(tf.truncated_normal(
      [num_hidden, num_labels], stddev=0.1))
  layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
  
  # Model.
  def model(data):
    #layer 1
    conv_1 = conv2d( data, layer1_weights, layer1_biases )
    pool_1 = maxpooling2d(conv_1, pool_size)
    
    # layer 2
    conv_2 = conv2d( pool_1  , layer2_weights, layer2_biases )
    pool_2 = maxpooling2d(conv_2, pool_size)

    # reshape
    shape = pool_2.get_shape().as_list()
    reshape = tf.reshape( pool_2, [shape[0], shape[1] * shape[2] * shape[3]])
    hidden = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
    return tf.matmul(hidden, layer4_weights) + layer4_biases
  
  # Training computation.
  logits = model(tf_train_dataset)
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
    
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
  test_prediction = tf.nn.softmax(model(tf_test_dataset))

#### run

In [12]:
num_steps = 1001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print('Initialized')
  for step in range(num_steps):
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run( [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 50 == 0):
      print('Minibatch loss at step %d: %f' % (step, l))
      print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 3.270008
Minibatch accuracy: 12.5%
Validation accuracy: 11.2%
Minibatch loss at step 50: 1.357963
Minibatch accuracy: 68.8%
Validation accuracy: 51.7%
Minibatch loss at step 100: 1.007636
Minibatch accuracy: 56.2%
Validation accuracy: 66.5%
Minibatch loss at step 150: 0.555476
Minibatch accuracy: 75.0%
Validation accuracy: 74.7%
Minibatch loss at step 200: 1.009805
Minibatch accuracy: 75.0%
Validation accuracy: 78.2%
Minibatch loss at step 250: 1.250130
Minibatch accuracy: 56.2%
Validation accuracy: 78.7%
Minibatch loss at step 300: 0.328498
Minibatch accuracy: 87.5%
Validation accuracy: 81.0%
Minibatch loss at step 350: 0.511847
Minibatch accuracy: 93.8%
Validation accuracy: 79.9%
Minibatch loss at step 400: 0.193068
Minibatch accuracy: 100.0%
Validation accuracy: 81.0%
Minibatch loss at step 450: 0.627322
Minibatch accuracy: 87.5%
Validation accuracy: 79.1%
Minibatch loss at step 500: 0.782499
Minibatch accuracy: 81.2%
Validation accuracy: 82.1%


___

#### Problem 2
Try to get the best performance you can using a convolutional net. Look for example at the classic LeNet5 architecture, adding Dropout, and/or adding learning rate decay.

In [24]:
# adding Dropout at layer 2

mp_layer_dropout    = 0.25
#mp_layer_dropout    = 0.10

dense_layer_dropout = 0.5

In [27]:
batch_size = 16
patch_size =  5
depth      = 16
num_hidden = 64

graph = tf.Graph()

with graph.as_default():

  # Input data.
  tf_train_dataset = tf.placeholder( tf.float32, shape=(batch_size, image_size, image_size, num_channels))
     
    
  tf_train_labels  = tf.placeholder( tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset  = tf.constant(test_dataset)
  
  # Variables.
  layer1_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, num_channels, depth], stddev=0.1))
  layer1_biases  = tf.Variable(tf.zeros([depth]))

  layer2_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, depth, depth], stddev=0.1))
  layer2_biases  = tf.Variable(tf.constant(1.0, shape=[depth]))

  layer3_weights = tf.Variable(tf.truncated_normal(
      [image_size // 4 * image_size // 4 * depth, num_hidden], stddev=0.1))
  layer3_biases  = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
  layer4_weights = tf.Variable(tf.truncated_normal(
      [num_hidden, num_labels], stddev=0.1))
  layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
  
  # Model.
  def model(data):
    #layer 1
    conv_1 = conv2d( data, layer1_weights, layer1_biases )
    #pool_1 = maxpooling2d(conv_1, pool_size)
    #81 81 87
    pool_1 = avgpooling2d(conv_1, pool_size)
    
    #pool_1 = tf.nn.dropout(pool_1, 1- mp_layer_dropout)
    
    # layer 2
    conv_2 = conv2d( pool_1  , layer2_weights, layer2_biases )
    #pool_2 = maxpooling2d(conv_2, pool_size)
    pool_2 = avgpooling2d(conv_2, pool_size)
    
    #pool_2 = tf.nn.dropout(pool_2, 1- mp_layer_dropout)

    # reshape
    shape = pool_2.get_shape().as_list()
    reshape = tf.reshape( pool_2, [shape[0], shape[1] * shape[2] * shape[3]])
    #hidden = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
    #return tf.matmul(hidden, layer4_weights) + layer4_biases

    '''dense_1 = dense( reshape, layer3_weights, layer3_biases )
    dense_2 = dense( dense_1, layer4_weights, layer4_biases )
    dense_2 = tf.nn.dropout(dense_2, 1 - dense_layer_dropout )
    return dense_2
    '''
    hidden = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
    layer_4 = tf.matmul(hidden, layer4_weights) + layer4_biases
    tf.nn.dropout( layer_4, 1 - dense_layer_dropout )
    return layer_4
    
    

  
  # Training computation.
  logits = model(tf_train_dataset)
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
    
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
  test_prediction = tf.nn.softmax(model(tf_test_dataset))

#### run p2

In [28]:
num_steps = 1001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print('Initialized')
  for step in range(num_steps):
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run( [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 50 == 0):
      print('Minibatch loss at step %d: %f' % (step, l))
      print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 3.989923
Minibatch accuracy: 6.2%
Validation accuracy: 10.3%
Minibatch loss at step 50: 2.138714
Minibatch accuracy: 12.5%
Validation accuracy: 32.7%
Minibatch loss at step 100: 1.266084
Minibatch accuracy: 50.0%
Validation accuracy: 59.5%
Minibatch loss at step 150: 0.675436
Minibatch accuracy: 81.2%
Validation accuracy: 73.7%
Minibatch loss at step 200: 0.990104
Minibatch accuracy: 62.5%
Validation accuracy: 74.7%
Minibatch loss at step 250: 1.414274
Minibatch accuracy: 62.5%
Validation accuracy: 77.8%
Minibatch loss at step 300: 0.451228
Minibatch accuracy: 87.5%
Validation accuracy: 79.3%
Minibatch loss at step 350: 0.648034
Minibatch accuracy: 93.8%
Validation accuracy: 77.3%
Minibatch loss at step 400: 0.287595
Minibatch accuracy: 100.0%
Validation accuracy: 80.4%
Minibatch loss at step 450: 0.839303
Minibatch accuracy: 75.0%
Validation accuracy: 79.9%
Minibatch loss at step 500: 0.836833
Minibatch accuracy: 87.5%
Validation accuracy: 80.4%
M